In [1]:
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph
import json
import random

In [2]:
nodes_path = "../file_data/fb-pages-food.nodes"
edges_path= "../file_data/fb-pages-food.edges"

nodes_df = pd.read_csv(
    nodes_path, sep=",", header=None, names=["node", "name", "new_id"], skiprows=1
)
edges_df = pd.read_csv(edges_path, sep=",", header=None, names=["source", "target"])

nodes_df["new_id"] = nodes_df["new_id"].astype(str)
edges_df["source"] = edges_df["source"].astype(str)
edges_df["target"] = edges_df["target"].astype(str)

In [3]:
G = nx.Graph()

for index, row in nodes_df.iterrows():
    G.add_node(row["new_id"], name=row["name"], original_id=row["node"])


for index, row in edges_df.iterrows():
    G.add_edge(row["source"], row["target"])

In [4]:
max_cliques = list(nx.find_cliques(G))
print(len(max_cliques))

1249


In [5]:
def generate_community_names(num_cliques):
    prefixes = [
        "Neo",
        "Alt",
        "Hyper",
        "Meta",
        "Quantum",
        "Terra",
        "Cyber",
        "Omega",
        "Solar",
        "Astra",
    ]
    roots = [
        "Gen",
        "Net",
        "Sphere",
        "Cluster",
        "Link",
        "Wave",
        "Flow",
        "Pulse",
        "Code",
        "Echo",
    ]
    suffixes = ["ium", "on", "os", "ix", "ia", "or", "an", "is", "us", "er"]

    community_names = set()

    while len(community_names) < num_cliques:
        name = (
            f"{random.choice(prefixes)}{random.choice(roots)}{random.choice(suffixes)}"
        )
        name += f" {random.randint(1000, 9999)}"
        if(name not in community_names):
            community_names.add(name)
    print("nomi generati")
    return list(community_names)

In [6]:
def assign_clique_names_to_graph(graph, cliques):
    num_cliques = len(cliques) 
    community_names = generate_community_names(
        num_cliques
    )
    for i, clique in enumerate(cliques):
        clique_name = community_names[i]
        saved_clique = clique
        cliques[i] = {"name": f"cliques {clique_name}" , "nodes": clique}
        for node in saved_clique:
            if not "aggregation" in graph.nodes[node]:
                graph.nodes[node]["aggregation"] = []
            graph.nodes[node]["aggregation"].append(f"cliques {clique_name}")
            if "community_name" not in graph.nodes[node]:
                graph.nodes[node]["community_name"] = clique_name
    for edge in graph.edges:
        source = edge[0]
        graph.edges[edge]["community_name"] = graph.nodes[source]["community_name"]
    print("nomi assegnati")

In [7]:
assign_clique_names_to_graph(G, max_cliques)
for node in G.nodes(data=True):
    print(f"Node {node[0]}: {node[1]['community_name']}")

nomi generati
nomi assegnati
Node 386: AstraEchoix 8542
Node 473: SolarSphereium 4290
Node 1: HyperClusterium 6268
Node 542: MetaPulseon 2867
Node 189: SolarEchoia 2126
Node 454: HyperLinkis 6349
Node 158: AstraNetos 7981
Node 248: SolarLinkos 8049
Node 537: SolarSphereer 5188
Node 293: TerraClusterer 6184
Node 171: HyperEchoos 1062
Node 485: AltLinkus 2353
Node 258: AstraNetos 5654
Node 534: OmegaFlowon 1250
Node 500: MetaEchoon 6539
Node 194: NeoCodeer 3740
Node 34: SolarPulseia 2264
Node 465: AltSphereium 2353
Node 562: TerraSphereium 6550
Node 511: QuantumWaveia 6455
Node 201: AltEchoia 1467
Node 188: CyberCodean 7510
Node 440: MetaNetia 9310
Node 484: AstraCodeos 2071
Node 240: AstraCodeos 2071
Node 533: OmegaPulseos 1324
Node 235: HyperEchoon 7911
Node 82: AltClusterium 7863
Node 212: CyberEchoer 2278
Node 35: TerraClusteran 9804
Node 19: HyperPulseis 8663
Node 154: OmegaPulseos 1324
Node 334: QuantumPulseia 9365
Node 148: MetaFlowan 8193
Node 599: NeoWaveor 4574
Node 382: MetaGe

In [8]:
graph_data = json_graph.node_link_data(G)

output_data= {
    "graph_data": graph_data,
    "max_cliques": max_cliques
}

In [9]:
with open("../graph_data/graph_data_cliques.json", "w") as f:
    json.dump(output_data, f, indent=4)